In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import os

In [ ]:
#function to split the data into training, validation and test sets, and calculate their Fourier coefficients along
#with the corresponding labels. At t=0 all of the signals are equal, so we omit the first timestep. Ntrain is the 
#number of trajectories in the training set, Nval is the number in the validation set and Ntest is the number in 
#the test set
def fouriertrainvaltest(X, Y, Ntrain, Nval, Ntest):
    
    #Generating a training set with Ntrain trajectories, a validation with Nval trajectories and a test set with
    #Ntest trajectories. At t=0 all of the signals are equal, so we omit the first timestep.
    Xtrain = X[0:Ntrain, 1:401]
    Xval = X[Ntrain:Ntrain+Nval, 1:401]
    Xtest = X[Ntrain+Nval:Ntrain+Nval+Ntest, 1:401]

    #extract the corresponding labels for the training, validation and test sets.
    Ytrain = Y[0:Ntrain, :]
    Yval = Y[Ntrain:Ntrain+Nval, :]
    Ytest = Y[Ntrain+Nval:Ntrain+Nval+Ntest, :]
    
    #calculating the Fourier coefficients for each subset.
    XtrainF = np.fft.fft(Xtrain)
    XvalF = np.fft.fft(Xval)
    XtestF = np.fft.fft(Xtest)
    
    #Prepare to split the Fourier coefficients into their real and imaginary components. Each complex number will 
    #occupy two columns: one for the real part and one for the imaginary part. Therefore, we create new arrays that 
    #have twice the number of columns. 
    xtrain = np.zeros((XtrainF.shape[0], 2*XtrainF.shape[1]))
    xval = np.zeros((XvalF.shape[0], 2*XvalF.shape[1]))
    xtest = np.zeros((XtestF.shape[0], 2*XtestF.shape[1]))

    #For each Fourier coefficient in the training set, split into real and imaginary parts. These parts are then
    #stored alternately (even indices for real, odd indices for imaginary).
    for i in range(XtrainF.shape[0]):
        for j in range(XtrainF.shape[1]):
            xtrain[i, 2*j] = XtrainF[i,j].real
            xtrain[i, 2*j + 1] = XtrainF[i,j].imag
        
    #Do the same for the test set, splitting the Fourier coefficients into their real and imaginary parts.
    for i in range(XtestF.shape[0]):
        for j in range(XtestF.shape[1]):
            xtest[i, 2*j] = XtestF[i,j].real
            xtest[i, 2*j + 1] = XtestF[i,j].imag
            
    #Similarly, split the Fourier coefficients for the validation set.
    for i in range(XvalF.shape[0]):
        for j in range(XvalF.shape[1]):
            xval[i, 2*j] = XvalF[i,j].real
            xval[i, 2*j + 1] = XvalF[i,j].imag
            
    #Return the transformed training, validation and test sets along with their corresponding labels
    return(xtrain, xval, xtest, Ytrain, Yval, Ytest)

In [ ]:
#Function to calculate the R-squared metric. This function takes the true and predicted values, and calculates the 
#R-squared value
def r_square(y, y_pred):
    residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
    total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
    r2 = 1 - residual/total
    return(r2)

In [ ]:
#Loading the datasets 
X = np.loadtxt('Data/Xtrainx.csv', delimiter=',')
Y = np.loadtxt('Data/Ytrain.csv', delimiter=',')

In [ ]:
#Extract the values of $\omega_c$, $s$ and $\eta$ from Y for training
Y_params = Y[:,3:6]

#Scale the extracted parameters to a [0,1] range using min-max scaling
scaler=MinMaxScaler()
Y_params_scaled = scaler.fit_transform(Y_params)

#Generating a training, validation and test set
xtrain, xval, xtest, Ytrain, Yval, Ytest = fouriertrainvaltest(X, Y_params_scaled, 1500, 300, 300)

In [ ]:
#define the path where the model's weights will be saved
checkpoint_path = "training.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
#Create a sequential model
model = tf.keras.Sequential()
#Add the hidden layers
model.add(tf.keras.layers.Dense(250, input_dim = (np.shape(xtrain)[1]), activation='sigmoid'))
model.add(tf.keras.layers.Dense(250, input_dim = (np.shape(xtrain)[1]), activation='sigmoid'))
model.add(tf.keras.layers.Dense(250, input_dim = (np.shape(xtrain)[1]), activation='sigmoid'))
model.add(tf.keras.layers.Dense(80 , activation = 'sigmoid'))
#Add the output layer
model.add(tf.keras.layers.Dense(3, activation = 'linear'))

#setting the optimiser equal to the Adam optimiser with learning rate = 0.0001
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

#compliling the model
model.compile(optimizer=opt, loss = 'mean_absolute_error', metrics=[r_square])

#training the model
history = model.fit(xtrain, Ytrain, epochs = 100000, validation_data = (xval, Yval), batch_size = np.shape(xtrain)[0], verbose=2, callbacks=[cp_callback])

In [ ]:
#Evaluating the model on the test set
model.evaluate(xtest, Ytest)

In [ ]:
#Evaluate the model on the training set
model.evaluate(xtrain, Ytrain)

In [ ]:
#configure font properties for the plots
plt.rc('font',family='Times New Roman')
plt.rcParams.update({'font.size':13})
plt.rcParams['font.size'] = 13
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['axes.linewidth'] = 1

In [ ]:
#inverse transform the scaled outputs to get original values
y_true = scaler.inverse_transform(Ytest)
y_pred = scaler.inverse_transform(model.predict(xtest))

In [ ]:
#Create a scatter plot of predicted values for $\eta$ versus true values
plt.plot(y_true[:,0], y_pred[:,0], 'bo', label='Predictions')
plt.plot(y_true[:,0], y_true[:,0], color='orange', label='True')
plt.legend()
plt.ylabel(r"$\mathrm{\hat{\eta} / \omega_0}$")
plt.xlabel(r"$\mathrm{\eta / \omega_0}$")
plt.savefig('ADpredictedetavreal.pdf', bbox_inches='tight')

In [ ]:
#define an array where each element is the difference between the predicted value and true value for $|eta$ for a 
#given trajectory
diffseta = y_pred[:,0] - y_true[:,0]

#print the minimum difference to find the smallest prediction error
print(np.amin(diffseta))
#print the maximum difference to find the largest prediction error
print(np.amax(diffseta))

In [ ]:
#define intervals for analysing the differences between predicted and true values for $\eta$
intervalseta = [-0.03, -0.029, -0.028, -0.027, -0.026, -0.025, -0.024, -0.023, -0.022, -0.021, -0.02, -0.019, -0.018, -0.017, -0.016, -0.015, -0.014, -0.013, -0.012, -0.011, -0.01, -0.009, -0.008, -0.007, -0.006, -0.005, -0.004, -0.003, -0.002, -0.001, -0.00, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016]

In [ ]:
#initialise list to store the frequencies of differences in intervals for $|eta$
freqeta = []

#loop through each interval
for i in range(len(intervalseta)-1):
    #create a mask to find differences within the current interval
    masketa = (diffseta >= intervalseta[i]) & (diffseta < intervalseta[i+1])
    #append the count of differences within the current interval to the frequency list
    freqeta.append(len(diffseta[masketa]))

In [ ]:
#calculate x positions for bars and heights
xeta = intervalseta[1:] 
heightseta = (np.array(freqeta)/len(y_true))*100

#create the bar chart
plt.bar(xeta, heightseta, width=-0.001, align='edge', alpha = 1, color = 'orange', edgecolor='black')  
plt.xlabel(r"$\mathrm{(\hat{\eta} - \eta)/\omega_0}$")
plt.ylabel('% of trajectories')
plt.savefig('ADbarcharteta.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#Create a scatter plot of predicted values for $\omega_c$ versus true values
plt.plot(y_true[:,1], y_pred[:,1], 'bo', label='Predictions')
plt.plot(y_true[:,1], y_true[:,1], color='orange', label='True')
plt.ylabel(r"$\mathrm{\hat{\omega}_c / \omega_0}$")
plt.xlabel(r"$\mathrm{\omega_c / \omega_0}$")
plt.legend()
plt.savefig('ADpredictedomegacvreal.pdf', bbox_inches='tight')

In [ ]:
#define an array where each element is the difference between the predicted value and true value for $|omega_c$ for
#a given trajectory
diffsomegac = y_pred[:,1] - y_true[:,1]

#print the minimum difference to find the smallest prediction error
print(np.amin(diffsomegac))
#print the maximum difference to find the largest prediction error
print(np.amax(diffsomegac))

In [ ]:
#define intervals for analysing the differences between predicted and true values for $\omega_c$
intervalsomegac = [-0.16, -0.15, -0.14, -0.13, -0.12, -0.11, -0.10, -0.09, -0.08, -0.07, -0.06, -0.05, -0.04, -0.03, -0.02, -0.01, 0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36]

In [ ]:
#initialise list to store the frequencies of differences in intervals for $|omega_c$
freqomegac = []

#loop through each interval
for i in range(len(intervalsomegac)-1):
    #create a mask to find differences within the current interval
    maskomegac = (diffsomegac >= intervalsomegac[i]) & (diffsomegac < intervalsomegac[i+1])
    #append the count of differences within the current interval to the frequency list
    freqomegac.append(len(diffsomegac[maskomegac]))

In [ ]:
#calculate x positions for bars and heights
xomegac = intervalsomegac[1:] 
heightsomegac = (np.array(freqomegac)/len(y_true))*100

#create the bar chart
plt.bar(xomegac, heightsomegac, width=-0.01, align='edge', alpha = 1, color = 'orange', edgecolor='black')  
plt.xlabel(r"$\mathrm{(\hat{\omega}_c - \omega_c)/\omega_0}$")
plt.ylabel('% of trajectories')
plt.savefig('ADbarchartomegac.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#Create a scatter plot of predicted values for $s$ versus true values
plt.plot(y_true[:,2], y_pred[:,2], 'bo', label='Predictions')
plt.plot(y_true[:,2], y_true[:,2], color='orange', label='True')
plt.ylabel(r"$\mathrm{\hat{s}}$")
plt.xlabel(r"$\mathrm{ s }$")
plt.legend()
plt.savefig('ADpredictedsvreal.pdf', bbox_inches='tight')

In [ ]:
#define an array where each element is the difference between the predicted value and true value for $s$ for
#a given trajectory
diffss = y_pred[:,2] - y_true[:,2]

#print the minimum difference to find the smallest prediction error
print(np.amin(diffss))
#print the maximum difference to find the largest prediction error
print(np.amax(diffss))

In [ ]:
#define intervals for analysing the differences between predicted and true values for $s$
intervalss = [-0.57, -0.56, -0.55, -0.54, -0.53, -0.52, -0.51, -0.5, -0.49, -0.48, -0.47, -0.46, -0.45, -0.44, -0.43, -0.42, -0.41, -0.4, -0.39, -0.38, -0.37, -0.36, -0.35, -0.34, -0.33, -0.32, -0.31, -0.3, -0.29, -0.28, -0.27, -0.26, -0.25, -0.24, -0.23, -0.22, -0.21, -0.2, -0.19, -0.18, -0.17, -0.16, -0.15, -0.14, -0.13, -0.12, -0.11, -0.10, -0.09, -0.08, -0.07, -0.06, -0.05, -0.04, -0.03, -0.02, -0.01, 0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12]

In [ ]:
#initialise list to store the frequencies of differences in intervals for $s$
freqs = []

#loop through each interval
for i in range(len(intervalss)-1):
    #create a mask to find differences within the current interval
    masks = (diffss >= intervalss[i]) & (diffss < intervalss[i+1])
    #append the count of differences within the current interval to the frequency list
    freqs.append(len(diffss[masks]))

In [ ]:
#calculate x positions for bars and heights
xs = intervalss[1:] 
heightss = (np.array(freqs)/len(y_true))*100

#create the bar chart
plt.bar(xs, heightss, width=-0.01, align='edge', alpha = 1, color = 'orange', edgecolor='black')  
plt.xlabel(r"$\mathrm{(\hat{\eta} - \eta)/\omega_0}$")
plt.ylabel('% of trajectories')
plt.savefig('ADbarcharts.pdf', bbox_inches='tight')
plt.show()